## DSCI 552 : HomeWork 7-PartA

### Name: Jayantraj CS (USC ID: 3993-1362-35)

In [ ]:
# Import Statements

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import warnings
import seaborn as sns
import random
import math
import time as t
import tensorflow as tf
from google.colab import drive
from tensorflow import keras
import tensorflow.keras.backend as k
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from skimage.color import rgb2gray
from tensorflow.keras.layers import Dense,Dropout,LSTM
from keras.utils import np_utils


drive.mount('/content/drive')
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## *Question 1.) Generative Models for Text*

## *Question 1c.) LSTM: Train an LSTM to mimic Russell’s style and thoughts*

## *Question 1c.i.) Concatenate your text files to create a corpus of Russell’s writings*

In [ ]:
# gets all the file names in the given path.
file_path='/content/drive/MyDrive/Colab Notebooks/HomeWork7/data/books/'
file_names=[files for root,directories,files in os.walk(file_path)][0]
corpus=[]
for file in file_names:
    b=open(file_path+ file, encoding='ascii', errors='ignore')
    corpus.append(b.read().lower())

## *Question 1c.ii.) Use a character-level representation for this model by using extended ASCII that has N = 256 characters*

In [ ]:
def character_representation(corpus_):
    unique_chars=set()
    char_ascii,ascii_char={},{}
    max_ascii =-float('inf')
    for book in corpus_:
        unique_chars.update(list(set(book)))
    total_num_of_uniq_chars=len(unique_chars)
    print('Total number of unique characters :',total_num_of_uniq_chars)
    for j in list(unique_chars):
        #print(j,ord(j))
        max_ascii=max(max_ascii,ord(j))
        char_ascii[j]=ord(j)
        ascii_char[ord(j)]=j
    return char_ascii,ascii_char,max_ascii

char_ascii,ascii_char,max_ascii=character_representation(corpus[:5])
    

Total number of unique characters : 68


Note: Since this is a large corpus for my computer’s power and it makes training LSTM hard(even with google colab pro), I will be using 5 books.

## *Question 1c.iii.) & 1c.iv.) Choose a window size, e.g., W = 100*

In [ ]:
def choose_window(book_):
    in_,out = [],[]
    for i in range(0, len(book_)-100+1,1):
        in_seq = book_[i : i + 100 - 1]
        out_seq = book_[i + 100 - 1]
        seq,val=[char_ascii[j] for j in in_seq],char_ascii[out_seq]
        in_.append(seq)
        out.append(val)
    return in_, out,seq,val

In [ ]:
def fun1(corpus_):
    seq_in,seq_out=[],[]
    for book in corpus_:
        cur_in, cur_out,seq,val = choose_window(book)
        seq_in.extend(cur_in)
        seq_out.extend(cur_out)
    return seq_in,seq_out

In [ ]:
start=t.time()
seq_in,seq_out = fun1(corpus[:5])
#print(len(seq_in))
end=t.time()
print("Time Taken for Execution:",end-start)

Time Taken for Execution: 31.73745346069336


## *Question 1c.v.) Rescaling or Normalizing the input and One-Hot encode the output*

In [ ]:
def return_one_hot_encode_array(seq_in,seq_out,max_ascii):
  input_lstm = np.reshape(seq_in,(len(seq_in),100-1,1))
  input_lstm = input_lstm / max_ascii
  output_lstm = np_utils.to_categorical(seq_out)
  return input_lstm,output_lstm

In [ ]:
start=t.time()
input_lstm,output_lstm=return_one_hot_encode_array(seq_in,seq_out,max_ascii)
print('The shape of the input :',input_lstm.shape)
print('The shape of the output :',output_lstm.shape)
end=t.time()
print("Time Taken for Execution:",end-start)

The shape of the input : (2683450, 99, 1)
The shape of the output : (2683450, 127)
Time Taken for Execution: 45.59769654273987


In [ ]:
# Verifying whether the input is normalized.
# We can see that the values are rescaled between [0,1].
print(input_lstm[0][:5])

[[0.78571429]
 [0.82539683]
 [0.76984127]
 [0.88888889]
 [0.92063492]]


In [ ]:
print(min(seq_out),max(seq_out))

10 126


We can see that the maximum ASCII value for the output vector is 126. This means after one-hot encoding the output vector will be a vector of size 127(0-126). Any one of the indexes 0-126 will become 1 if it corresponds to the character. This justifies, why our ouput_lstm vector has a shape (2683450, 127).

## *Questions 1c.vi - c.x.) Building the LSTM model*

In [ ]:
model_lstm = Sequential()
x,y=input_lstm.shape[1],input_lstm.shape[2]

In [ ]:
model_lstm.add(LSTM(256,input_shape=(x,y)))
model_lstm.add(Dropout(0.2))


In [ ]:
model_lstm.add(Dense(output_lstm.shape[1], activation='softmax'))
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam')

 

*   Log loss or Cross-entropy as the objective function of the network. Log loss and Cross-entropy are essentially the same. Typically, we will use the term log loss for binary classification problems, and cross-entropy loss for multiclass-classification problems. But they can be used synonymously.

*   cross-entropy loss measures the performance of a classification model whose output is a probability value between 0 and 1.

*   When we are adjusting or changing weights during training(like our case), cross-entropy loss is used. The goal is to reduce or minimize the loss, the model which has the smallest loss is the better model.






In [ ]:
print(model_lstm.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 256)               264192    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 127)               32639     
                                                                 
Total params: 296,831
Trainable params: 296,831
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
weights_file_path='/content/drive/MyDrive/Colab Notebooks/HomeWork7'
cp = ModelCheckpoint(weights_file_path+"/weights/loss_value{loss:}.hdf5", monitor='loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
start =t.time()
model_lstm.fit(input_lstm, output_lstm, epochs=30, batch_size=64, callbacks=[cp])
end=t.time()
print("Time Taken for Execution:",end-start)

Epoch 1/30
41926/41929 [============================>.] - ETA: 0s - loss: 2.6672
Epoch 00001: loss improved from inf to 2.66720, saving model to /content/drive/MyDrive/Colab Notebooks/HomeWork7/weights/loss_value2.6672046184539795.hdf5
41929/41929 [==============================] - 484s 11ms/step - loss: 2.6672
Epoch 2/30
41925/41929 [============================>.] - ETA: 0s - loss: 2.3422
Epoch 00002: loss improved from 2.66720 to 2.34214, saving model to /content/drive/MyDrive/Colab Notebooks/HomeWork7/weights/loss_value2.3421361446380615.hdf5
41929/41929 [==============================] - 479s 11ms/step - loss: 2.3421
Epoch 3/30
41926/41929 [============================>.] - ETA: 0s - loss: 2.1642
Epoch 00003: loss improved from 2.34214 to 2.16418, saving model to /content/drive/MyDrive/Colab Notebooks/HomeWork7/weights/loss_value2.164184093475342.hdf5
41929/41929 [==============================] - 478s 11ms/step - loss: 2.1642
Epoch 4/30
41926/41929 [============================>.

## *Question 1c.xi.) Use the network with the best weights to generate 1000 characters, using the given text.*

In [ ]:
def get_the_ascii_list(text):
  result=[]
  for i in text:
    result.append(char_ascii[i])
  return result

text ='There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.'
ascii_text=get_the_ascii_list(text[-99:].lower())


In [ ]:
new_text=0
for i in range(1000):
  r=len(ascii_text)
  new_text = np.reshape(ascii_text, (1, r, 1))
  new_text = new_text / max_ascii
  character_predicted = model_lstm.predict(new_text, verbose=0)
  index_predicted = np.argmax(character_predicted)
  text=text+ ascii_char[index_predicted]
  ascii_text.append(index_predicted)
  q=len(ascii_text)
  ascii_text = ascii_text[1:q]
print(text)

There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.
the semsetive perception of the semse dnrsel to the semsetive poonssi
of the semsetive poonssi of the semsetive poonssi of the semsetive poonss
of the semse- and thete are the semsetian aosteee of the semsetive poonss
of the semse- and thete are the semsetian aosteee of the semsetive poonss
of the semse- and thete are the semsetian aosteee of the semsetive poonss
of the semse- and thete are the semsetian aosteee of the semsetive poonss
of the semse- and thete are the semsetian aosteee of the semsetive poonss
of the semse- and thete are the semsetian aosteee of the semsetive poonss
of the semse- and thete are the semsetian aosteee of the semsetive poonss
of the semse- and thete are the semsetian aosteee of the semsetive poonss
of the semse- and thete are the semsetian aosteee of the semsetive poonss
of the semse- and thete are the sems